In [66]:

import numpy as np
import cv2
import os
import sys
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import f1_score



In [67]:
# Hyperparameters and settings
batch_size = 64
test_batch_size = 64
input_size = 1 # because there is only one channel 
output_size = 4
num_epochs = 10
learning_rate = 0.001



In [68]:
# Load traiing, validation and training data

data_loader = torch.load('data_loader.pt')
valid_loader = torch.load('valid_loader.pt')
test_loader = torch.load('test_loader.pt')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the data loaders to the desired device
data_loader.dataset.tensors = tuple(tensor.to(device) for tensor in data_loader.dataset.tensors)
valid_loader.dataset.tensors = tuple(tensor.to(device) for tensor in valid_loader.dataset.tensors)
test_loader.dataset.tensors = tuple(tensor.to(device) for tensor in test_loader.dataset.tensors)


In [69]:
class CNN(nn.Module):
    def __init__(self):
        self.name = "CNN"
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(12 * 12 * 64, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x



In [70]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.name = "CNN2"
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            #get dimensions of last layer
            
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(64*9*9, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x

In [71]:
class CNN3(nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.name = "CNN3"
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=7, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
           
            
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(20*20*64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x

In [75]:
def train_and_save_model(model, data_loader, valid_loader, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_step = len(data_loader)
    loss_list = []
    acc_list = []
    f1_list = []

    loss_list_test = []
    acc_list_test = []
    f1_list_test = []
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(data_loader):

            # Move images and labels to the device
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss_list.append(loss.item())
            # Backprop and optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Train accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)
            # Train F1 score
            f1 = f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')
            f1_list.append(f1)
            # print(i)
            # if (i + 1) % 10 == 0:
            #     print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
            #     .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
            #     (correct / total) * 100))

        with torch.no_grad():
            for i, (images_test, labels_test) in enumerate(valid_loader):
                images_test = images_test.to(device)
                labels_test = labels_test.to(device)
                outputs_test = model(images_test)
                loss_test = criterion(outputs_test, labels_test)
                loss_list_test.append(loss_test.item())
                total_test = labels_test.size(0)
                _, predicted_test = torch.max(outputs_test.data, 1)
                correct_test = (predicted_test == labels_test).sum().item()
                acc_list_test.append(correct_test / total_test)

                #Test F1 score
                f1_test = f1_score(labels_test.cpu().numpy(), predicted_test.cpu().numpy(), average='macro')
                f1_list_test.append(f1_test)
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, F1 Score: {:.2f}'
        .format(epoch + 1, num_epochs, i + 1, total_step, loss_test.item(),
        (correct / total) * 100, f1_test))
        
        print(loss_test.item())
        

    # Save model to file
    torch.save(model.state_dict(), '%s.pth'%(model.name))




In [77]:
model1 = CNN().to(device)
train_and_save_model(model1, data_loader, valid_loader, device)


Epoch [1/10], Step [10/44], Loss: 0.9008, Accuracy: 66.67%
0.9008370041847229
Epoch [2/10], Step [10/44], Loss: 0.5256, Accuracy: 83.33%
0.5256298780441284
Epoch [3/10], Step [10/44], Loss: 1.1318, Accuracy: 79.17%
1.1317824125289917
Epoch [4/10], Step [10/44], Loss: 0.1839, Accuracy: 83.33%
0.18392497301101685
Epoch [5/10], Step [10/44], Loss: 0.5536, Accuracy: 95.83%
0.553557813167572
Epoch [6/10], Step [10/44], Loss: 0.3379, Accuracy: 87.50%
0.3379003703594208
Epoch [7/10], Step [10/44], Loss: 0.5953, Accuracy: 91.67%
0.5953469276428223
Epoch [8/10], Step [10/44], Loss: 0.3537, Accuracy: 95.83%
0.3537135124206543
Epoch [9/10], Step [10/44], Loss: 0.0593, Accuracy: 100.00%
0.0592837929725647
Epoch [10/10], Step [10/44], Loss: 0.3104, Accuracy: 87.50%
0.3104085624217987


In [78]:
#device = torch.device('cpu')

model2 = CNN2().to(device)

train_and_save_model(model2, data_loader, valid_loader, device)


Epoch [1/10], Step [10/44], Loss: 0.3491, Accuracy: 91.67%
0.3491349518299103
Epoch [2/10], Step [10/44], Loss: 0.6591, Accuracy: 95.83%
0.6590884327888489
Epoch [3/10], Step [10/44], Loss: 0.3459, Accuracy: 87.50%
0.34587791562080383
Epoch [4/10], Step [10/44], Loss: 0.0287, Accuracy: 75.00%
0.028725184500217438
Epoch [5/10], Step [10/44], Loss: 0.3504, Accuracy: 95.83%
0.3503977060317993
Epoch [6/10], Step [10/44], Loss: 0.0724, Accuracy: 91.67%
0.07237362116575241
Epoch [7/10], Step [10/44], Loss: 0.3712, Accuracy: 95.83%
0.37121155858039856
Epoch [8/10], Step [10/44], Loss: 0.0116, Accuracy: 95.83%
0.011619550175964832
Epoch [9/10], Step [10/44], Loss: 0.2681, Accuracy: 91.67%
0.26814743876457214
Epoch [10/10], Step [10/44], Loss: 0.2143, Accuracy: 91.67%
0.21433930099010468


In [45]:

model3 = CNN3().to(device)
train_and_save_model(model3, data_loader, valid_loader, device)

Epoch [1/10], Step [10/44], Loss: 0.6144, Accuracy: 83.33%
0.6143677830696106
Epoch [2/10], Step [10/44], Loss: 0.4271, Accuracy: 75.00%
0.4270746409893036
Epoch [3/10], Step [10/44], Loss: 0.1920, Accuracy: 95.83%
0.19197624921798706
Epoch [4/10], Step [10/44], Loss: 0.4687, Accuracy: 95.83%
0.4686531722545624
Epoch [5/10], Step [10/44], Loss: 0.2541, Accuracy: 91.67%
0.2540655732154846
Epoch [6/10], Step [10/44], Loss: 0.5060, Accuracy: 87.50%
0.5059526562690735
Epoch [7/10], Step [10/44], Loss: 0.9441, Accuracy: 95.83%
0.9441371560096741
Epoch [8/10], Step [10/44], Loss: 0.4995, Accuracy: 95.83%
0.49953651428222656
Epoch [9/10], Step [10/44], Loss: 0.1159, Accuracy: 91.67%
0.1159004345536232
Epoch [10/10], Step [10/44], Loss: 2.3058, Accuracy: 95.83%
2.305830240249634


In [82]:
def evaluate_model(model, valid_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Test Accuracy of the model (validation): {} %'
        .format((correct / total) * 100))

def test_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Test Accuracy of the model (testing): {} %'
        .format((correct / total) * 100))

In [83]:
for model in [model1, model2, model3]:
    evaluate_model(model, valid_loader)
    test_model(model, test_loader)

Test Accuracy of the model (validation): 88.66666666666667 %
Test Accuracy of the model (testing): 89.66666666666666 %
Test Accuracy of the model (validation): 92.33333333333333 %
Test Accuracy of the model (testing): 92.66666666666666 %
Test Accuracy of the model (validation): 86.33333333333333 %
Test Accuracy of the model (testing): 89.0 %


In [52]:
# function to test an individual image from an external source
def test_individual_image(model, image_name, category, read_custom_path = ''):
    if (read_custom_path != ''):
        img = cv2.imread(read_custom_path, 0)
        image_name = read_custom_path
        #save the image
    else:
        img = cv2.imread("../concat_data/%s/%s" % (category, image_name), 0)
    
    #resize to 48x48 pixels
    img = cv2.resize(img, (48, 48))



    #if image 3 channel convert to 1 channel
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    cv2.imwrite("image_mod.jpg", img)
    
    # Convert image to tensor and add batch and channel dimensions
    img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    
    # Get label tensor
    labels = {'focused': 0, 'happy': 1, 'neutral': 2, 'surprised': 3}
    reverse_labels = {0: 'focused', 1: 'happy', 2: 'neutral', 3: 'surprised'}
    label_tensor = torch.tensor(labels[category], dtype=torch.long)

    # Forward pass for the single image
    img_tensor = img_tensor.to(device)
    output = model(img_tensor)
    _, predicted = torch.max(output, 1)

    # Print results
    print("Predicted:", predicted.item(), "(", reverse_labels[predicted.item()], ")")
    print("Actual:", label_tensor.item(), "(", reverse_labels[label_tensor.item()], ")")
    print("Image:", image_name)
    print("Category:", category)
    print("///////")
    
    return predicted.item()

In [60]:
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "happy", read_custom_path=r"C:\Users\Luis\Downloads\test_smile.PNG")


Predicted: 1 ( happy )
Actual: 1 ( happy )
Image: C:\Users\Luis\Downloads\test_smile.PNG
Category: happy
///////


1

In [61]:
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "neutral", read_custom_path=r"C:\Users\Luis\Downloads\test_neutral.PNG")

Predicted: 1 ( happy )
Actual: 2 ( neutral )
Image: C:\Users\Luis\Downloads\test_neutral.PNG
Category: neutral
///////


1

In [62]:
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"C:\Users\Luis\Downloads\test_focused.PNG")
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"C:\Users\Luis\Downloads\test_focused_2.PNG")
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"C:\Users\Luis\Downloads\test_focused_3.PNG")
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"C:\Users\Luis\Downloads\test_focused_4.PNG")
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused", read_custom_path=r"C:\Users\Luis\Downloads\test_focused_5.PNG")

Predicted: 3 ( surprised )
Actual: 0 ( focused )
Image: C:\Users\Luis\Downloads\test_focused.PNG
Category: focused
///////
Predicted: 3 ( surprised )
Actual: 0 ( focused )
Image: C:\Users\Luis\Downloads\test_focused_2.PNG
Category: focused
///////
Predicted: 0 ( focused )
Actual: 0 ( focused )
Image: C:\Users\Luis\Downloads\test_focused_3.PNG
Category: focused
///////
Predicted: 1 ( happy )
Actual: 0 ( focused )
Image: C:\Users\Luis\Downloads\test_focused_4.PNG
Category: focused
///////
Predicted: 0 ( focused )
Actual: 0 ( focused )
Image: C:\Users\Luis\Downloads\test_focused_5.PNG
Category: focused
///////


0

In [65]:
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "surprised", read_custom_path=r"C:\Users\Luis\Downloads\test_surprised.PNG")

Predicted: 3 ( surprised )
Actual: 3 ( surprised )
Image: ../unseen-test-imgs/test_surprised.PNG
Category: surprised
///////


3

In [59]:
test_individual_image(model1, "86_MMA-FACIAL-EXPRESSION-mahmoud.jpg", "focused")

[ WARN:0@1160.917] global loadsave.cpp:248 findDecoder imread_('../concat_data/focused/86_MMA-FACIAL-EXPRESSION-mahmoud.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
